Business question: What are the most significant predictors of student achievement on a school level?

In [1]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sn
#import functions as fn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, roc_auc_score, plot_confusion_matrix, precision_score, recall_score 
#from category_encoders import CountEncoder

In [2]:
#View options
pd.set_option('display.max_columns', None, 'display.float_format', lambda x: '%8.0f' % x, 'display.max_rows', None)

# Data Preparation and Cleaning

In [3]:
df = pd.read_csv('data/us_doe_school_characteristics.csv',encoding='cp1252')

In [4]:
df.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT
0,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction
1,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction
2,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,1,nan,AL-101,100005,AL-101-0110,10000500879,100879,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction
3,2020-2021,1,ALABAMA,AL,Albertville Elementary School,1,nan,AL-101,100005,AL-101-0200,10000500889,100889,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction
4,2020-2021,1,ALABAMA,AL,Albertville Kindergarten and PreK,1,nan,AL-101,100005,AL-101-0035,10000501616,101616,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction


In [5]:
df.shape

(99763, 20)

In [6]:
locations_df = pd.read_csv('data/us_doe_locations.csv', encoding = 'cp1252')

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (14,15,21,22,39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
locations_df.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2020-2021,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,nan,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,7,8,Middle,As reported
1,2020-2021,1,ALABAMA,AL,Albertville High School,Albertville City,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,9,12,High,As reported
2,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,Albertville City,1,nan,AL-101,100005,AL-101-0110,10000500879,100879,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300,(256)878-7698,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,5,6,Middle,As reported
3,2020-2021,1,ALABAMA,AL,Albertville Elementary School,Albertville City,1,nan,AL-101,100005,AL-101-0200,10000500889,100889,145 West End Drive,NaN,NaN,Albertville,AL,35950,2532,145 West End Drive,NaN,NaN,Albertville,AL,35950,nan,(256)894-4822,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,3,4,Elementary,As reported
4,2020-2021,1,ALABAMA,AL,Albertville Kindergarten and PreK,Albertville City,1,nan,AL-101,100005,AL-101-0035,10000501616,101616,257 Country Club Rd,NaN,NaN,Albertville,AL,35951,3927,257 Country Club Rd,NaN,NaN,Albertville,AL,35951,3927,(256)878-7922,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,PK,KG,Elementary,As reported


In [8]:
enrollment = pd.read_excel('data/USDOE_Enrollment.xlsx')

In [9]:
enrollment.shape

(99581, 4)

In [10]:
enrollment.head(10)

,ELSI Export,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,National Center for Education Statistics - htt...,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,This is a Public School based table with the f...,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,School Name,State Name [Public School] Latest available year,School ID - NCES Assigned [Public School] Late...,Total Students All Grades (Excludes AE) [Publi...
6,1 LT CHARLES W. WHITCOMB SCHOOL,Massachusetts,250732002639,1139
7,100 ACADEMY OF EXCELLENCE ES,Nevada,320006000670,352
8,100 ACADEMY OF EXCELLENCE MS,Nevada,320006000756,163
9,10TH STREET SCHOOL,Washington,530486002475,175


In [11]:
enrollment_fixed = enrollment.drop([0,1,2,3,4]).reset_index()

In [12]:
enrollment_fixed.head()

,index,ELSI Export,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,5,School Name,State Name [Public School] Latest available year,School ID - NCES Assigned [Public School] Late...,Total Students All Grades (Excludes AE) [Publi...
1,6,1 LT CHARLES W. WHITCOMB SCHOOL,Massachusetts,250732002639,1139
2,7,100 ACADEMY OF EXCELLENCE ES,Nevada,320006000670,352
3,8,100 ACADEMY OF EXCELLENCE MS,Nevada,320006000756,163
4,9,10TH STREET SCHOOL,Washington,530486002475,175


In [13]:
enrollment_fixed.columns = enrollment_fixed.iloc[0]

In [14]:
enrollment_fixed = enrollment_fixed[1:]

In [15]:
enrollment_fixed.head()

,5,School Name,State Name [Public School] Latest available year,School ID - NCES Assigned [Public School] Latest available year,Total Students All Grades (Excludes AE) [Public School] 2020-21
1,6,1 LT CHARLES W. WHITCOMB SCHOOL,Massachusetts,250732002639,1139
2,7,100 ACADEMY OF EXCELLENCE ES,Nevada,320006000670,352
3,8,100 ACADEMY OF EXCELLENCE MS,Nevada,320006000756,163
4,9,10TH STREET SCHOOL,Washington,530486002475,175
5,10,112 ALC INDEPENDENT STUDY,Minnesota,270819004415,34


In [16]:
new_enrollment = enrollment_fixed.rename(columns = {'School ID - NCES Assigned [Public School] Latest available year':'NCESSCH'})

In [17]:
new_enrollment.head()

,5,School Name,State Name [Public School] Latest available year,NCESSCH,Total Students All Grades (Excludes AE) [Public School] 2020-21
1,6,1 LT CHARLES W. WHITCOMB SCHOOL,Massachusetts,250732002639,1139
2,7,100 ACADEMY OF EXCELLENCE ES,Nevada,320006000670,352
3,8,100 ACADEMY OF EXCELLENCE MS,Nevada,320006000756,163
4,9,10TH STREET SCHOOL,Washington,530486002475,175
5,10,112 ALC INDEPENDENT STUDY,Minnesota,270819004415,34


In [18]:
teachers = pd.read_csv('data/us_doe_teachercount.csv', encoding = 'cp1252')

In [19]:
teachers.shape

(99763, 15)

In [20]:
teachers.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,NCESSCH,SCHID,TEACHERS,TOTAL_INDICATOR,DMS_FLAG
0,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,42,Education Unit Total,Reported
1,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,82,Education Unit Total,Reported
2,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,1,nan,AL-101,100005,AL-101-0110,10000500879,100879,41,Education Unit Total,Reported
3,2020-2021,1,ALABAMA,AL,Albertville Elementary School,1,nan,AL-101,100005,AL-101-0200,10000500889,100889,49,Education Unit Total,Reported
4,2020-2021,1,ALABAMA,AL,Albertville Kindergarten and PreK,1,nan,AL-101,100005,AL-101-0035,10000501616,101616,30,Education Unit Total,Reported


In [21]:
df.shape

(99763, 20)

In [22]:
locations_df.shape

(101662, 65)

In [23]:
locations_df.duplicated(subset = ['NCESSCH']).sum()

71596

In [24]:
locations_df = locations_df.drop_duplicates(subset = ['NCESSCH'])

In [ ]:
locations_df.loc[locations_df['']]

In [25]:
locations_df.shape

(30066, 65)

In [46]:
#Merge all national-level dataframes together
merged1 = df.merge(locations_df, how ='inner', on = 'NCESSCH')

In [47]:
merged1.shape

(99763, 85)

In [66]:
merged2 = new_enrollment.merge(teachers, how ='outer', on = 'NCESSCH')

In [67]:
merged2.shape

(119467, 19)

In [68]:
merged2['NCESSCH'] = merged2['NCESSCH'].astype(float)

In [60]:
merged1sorted = merged1.sort_values(by=['NCESSCH'], ascending=True)

In [61]:
merged1sorted.head()

,index,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH,SCHID_x,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,LEA_NAME,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,SCHID_y,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,0,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,nan,AL-101,100005,AL-101-0010,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,nan,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,7,8,Middle,As reported
1,1,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville High School,Albertville City,1,nan,AL-101,100005,AL-101-0020,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,9,12,High,As reported
2,2,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,1,nan,AL-101,100005,AL-101-0110,10000500879,100879,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,Albertville City,1,nan,AL-101,100005,AL-101-0110,100879,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300,901 W McKinney Ave,NaN,NaN,Albertville,AL,35950,1300,(256)878-7698,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,5,6,Middle,As reported
3,3,2020-2021,1,ALABAMA,AL,Albertville Elementary School,1,nan,AL-101,100005,AL-101-0200,10000500889,100889,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Elementary School,Albertville City,1,nan,AL-101,100005,AL-101-0200,100889,145 West End Drive,NaN,NaN,Albertville,AL,35950,2532,145 West End Drive,NaN,NaN,Albertville,AL,35950,nan,(256)894-4822,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,3,4,Elementary,As reported
4,4,2020-2021,1,ALABAMA,AL,Albertville Kindergarten and PreK,1,nan,AL-101,100005,AL-101-0035,10000501616,101616,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Kindergarten and PreK,Albertville City,1,nan,AL-101,100005,AL-101-0035,101616,257 Country Club Rd,NaN,NaN,Albertville,AL,35951,

In [69]:
merged2sorted = merged2.sort_values(by=['NCESSCH'], ascending=True)

In [70]:
merged2sorted.head()

,5,School Name,State Name [Public School] Latest available year,NCESSCH,Total Students All Grades (Excludes AE) [Public School] 2020-21,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,SCHID,TEACHERS,TOTAL_INDICATOR,DMS_FLAG
1325,1331,ALBERTVILLE MIDDLE SCHOOL,Alabama,10000500870,908,NaN,nan,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,NaN
99575,nan,NaN,NaN,10000500870,NaN,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,100870,42,Education Unit Total,Reported
1322,1328,ALBERTVILLE HIGH SCHOOL,Alabama,10000500871,1606,NaN,nan,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,NaN
99576,nan,NaN,NaN,10000500871,NaN,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,100871,82,Education Unit Total,Reported
99577,nan,NaN,NaN,10000500879,NaN,2020-2021,1,ALABAMA,AL,Albertville Intermediate School,1,nan,AL-101,100005,AL-101-0110,100879,41,Education Unit Total,Reported


In [74]:
us_df = pd.merge(merged1, merged2, how = 'outer', on = 'NCESSCH')

In [75]:
us_df.head()

,index,SCHOOL_YEAR_x,FIPST_x,STATENAME_x,ST_x,SCH_NAME_x,STATE_AGENCY_NO_x,UNION_x,ST_LEAID_x,LEAID_x,ST_SCHID_x,NCESSCH,SCHID_x,SHARED_TIME,TITLEI_STATUS,TITLEI_STATUS_TEXT,MAGNET_TEXT,NSLP_STATUS,NSLP_STATUS_TEXT,VIRTUAL,VIRTUAL_TEXT,SCHOOL_YEAR_y,FIPST_y,STATENAME_y,ST_y,SCH_NAME_y,LEA_NAME,STATE_AGENCY_NO_y,UNION_y,ST_LEAID_y,LEAID_y,ST_SCHID_y,SCHID_y,MSTREET1,MSTREET2,MSTREET3,MCITY,MSTATE,MZIP,MZIP4,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,LZIP4,PHONE,WEBSITE,SY_STATUS,SY_STATUS_TEXT,UPDATED_STATUS,UPDATED_STATUS_TEXT,EFFECTIVE_DATE,SCH_TYPE_TEXT,SCH_TYPE,RECON_STATUS,OUT_OF_STATE_FLAG,CHARTER_TEXT,CHARTAUTH1,CHARTAUTHN1,CHARTAUTH2,CHARTAUTHN2,NOGRADES,G_PK_OFFERED,G_KG_OFFERED,G_1_OFFERED,G_2_OFFERED,G_3_OFFERED,G_4_OFFERED,G_5_OFFERED,G_6_OFFERED,G_7_OFFERED,G_8_OFFERED,G_9_OFFERED,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED,5,School Name,State Name [Public School] Latest available year,Total Students All Grades (Excludes AE) [Public School] 2020-21,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,ST_SCHID,SCHID,TEACHERS,TOTAL_INDICATOR,DMS_FLAG
0,0,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,nan,AL-101,100005,AL-101-0010,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,nan,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,7,8,Middle,As reported,1331,ALBERTVILLE MIDDLE SCHOOL,Alabama,908,NaN,nan,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,NaN
1,0,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,10000500870,100870,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville Middle School,Albertville City,1,nan,AL-101,100005,AL-101-0010,100870,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,2336,600 E Alabama Ave,NaN,NaN,Albertville,AL,35950,nan,(256)878-2341,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,7,8,Middle,As reported,nan,NaN,NaN,NaN,2020-2021,1,ALABAMA,AL,Albertville Middle School,1,nan,AL-101,100005,AL-101-0010,100870,42,Education Unit Total,Reported
2,1,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville High School,Albertville City,1,nan,AL-101,100005,AL-101-0020,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regular School,1,No,No,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,9,12,High,As reported,1328,ALBERTVILLE HIGH SCHOOL,Alabama,1606,NaN,nan,NaN,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,NaN
3,1,2020-2021,1,ALABAMA,AL,Albertville High School,1,nan,AL-101,100005,AL-101-0020,10000500871,100871,No,SWELIGTGPROG,Title I schoolwide eligible-Title I targeted a...,No,NSLPWOPRO,Yes participating without using any Provision ...,NOTVIRTUAL,No virtual instruction,2020-2021,1,ALABAMA,AL,Albertville High School,Albertville City,1,nan,AL-101,100005,AL-101-0020,100871,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2399,402 E McCord Ave,NaN,NaN,Albertville,AL,35950,2322,(256)894-5000,http://www.albertk12.org,1,Open,1,Open,7/20/21,Regul

In [76]:
us_df.shape

(200492, 103)